In [1]:
from encodings.punycode import punycode_decode
from zipfile import ZipFile
import os

from numpy.ma.core import shape


def zip_process(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".zip"):
            file_path=os.path.join(folder_path, file_name)
            dir_name=os.path.splitext(file_path)[0]
            with ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(dir_name)
                

zip_process('./data/Sublabel')
zip_process('./data/Training/01.원천데이터')
zip_process('./data/Training/02.라벨링데이터')
zip_process('./data/Validation/01.원천데이터')
zip_process('./data/Validation/02.라벨링데이터')

In [62]:
import json
import os
# coding: utf-8
import sys
import numpy as np
sys.path.append('..')
from optimizer import SGD
from trainer import RnnlmTrainer
from util import eval_perplexity

In [58]:
def preprocess_json(data):
  """
    data:json format file
    xs:texts
    ts:evaluations
  """
  xs = []
  ts = []
  for conversation in data['dataset']['conversations']:
    for utterance in conversation['utterances']:
      if utterance['speaker_id'] == 0:
        xs.append(utterance['utterance_text'])
        ts.append(utterance['utterance_evaluation'])
  return xs, ts

In [59]:
folder_path='./data/Training/02.라벨링데이터/TL_2.대화단위평가_기술_과학' 
xs_list=[]
ts_list=[]
for filename in os.listdir(folder_path):
    #print(filename)
    if filename.endswith('.json'):
        #print(filename)
        file_path=os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
          data=json.load(f)
        xs,ts=preprocess_json(data)
        xs_list.extend(xs)
        ts_list.extend(ts)

In [60]:
xs_list[0]

"스트랜드는 '외가닥'이나 '실타래'라는 뜻으로, 최종 편집본을 한 가닥으로 끝까지 연결하는 것을 말합니다."

In [61]:
ts_list[0]

[{'linguistic_acceptability': 'yes',
  'consistency': 'yes',
  'interestingness': 'yes',
  'unbias': 'yes',
  'harmlessness': 'yes',
  'no_hallucination': 'yes',
  'understandability': 'yes',
  'sensibleness': 'yes',
  'specificity': 'yes'},
 {'linguistic_acceptability': 'yes',
  'consistency': 'yes',
  'interestingness': 'yes',
  'unbias': 'yes',
  'harmlessness': 'yes',
  'no_hallucination': 'yes',
  'understandability': 'yes',
  'sensibleness': 'yes',
  'specificity': 'yes'},
 {'linguistic_acceptability': 'yes',
  'consistency': 'yes',
  'interestingness': 'yes',
  'unbias': 'yes',
  'harmlessness': 'yes',
  'no_hallucination': 'yes',
  'understandability': 'yes',
  'sensibleness': 'yes',
  'specificity': 'yes'}]

In [63]:
def convert_number(ts):
    '''
    3개의 결과를 1개로 만들고, yes -> 1, no -> 0으로 변환
    :param ts: 
    :return: 
    '''
    result=[]
    for i in ts[0].keys():
        v=[1 if x[i]=='yes' else 0 for x in ts]
        result.append(int(np.mean(v)>=0.5))
    return result
ts_list=[convert_number(ts) for ts in ts_list]

In [64]:
word_to_id = {}
id_to_word = {}
def preprocess(text,word_to_id,id_to_word):
    text = text.lower()
    text = text.replace('.', ' .')
    text = text.replace(',', ' ,')
    text = text.replace('!', ' !')
    text = text.replace('?', ' ?')
    text = text.replace("'", ' ')
    text = text.replace('"', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('[', ' ')
    text = text.replace(']', ' ')
    words = text.split(' ')


    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus

In [65]:
# 하이퍼파라미터 설정
batch_size = 20
wordvec_size = 100
hidden_size = 100  # RNN의 은닉 상태 벡터의 원소 수
time_size = 35     # RNN을 펼치는 크기
lr = 20.0
max_epoch = 4
max_grad = 0.25

In [66]:
corpus_list = []
for i in xs_list:
    corpus = preprocess(i, word_to_id, id_to_word)
    corpus_list.append(corpus)

In [67]:
vocab_size = len(word_to_id)

In [68]:
vocab_size

215295

In [69]:
ts_list

[[1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 0, 0, 0, 1, 1],
 [0, 1, 1, 0, 0, 0, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 0, 0, 0, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1],
 [0, 1, 1,

In [70]:
xs_list

["스트랜드는 '외가닥'이나 '실타래'라는 뜻으로, 최종 편집본을 한 가닥으로 끝까지 연결하는 것을 말합니다.",
 '스트랜드 방식은 가장 간단하고 비용도 저렴하지만, 옵티컬 효과를 제대로 구사하기 힘들다는 단점이 있습니다.',
 '옵티컬 효과를 내려면 페이드나 디졸브 등의 효과를 위해 두 개의 롤을 중첩시켜야 합니다. 하지만 싱글 스트랜드는 하나의 롤만을 만들기 때문에 그런 효과를 직접 만들 수 없습니다.',
 '네, 16mm와 슈퍼 8mm에서 싱글 스트랜드는 필름과 필름을 이은 접착면이 영사를 할 때 문제가 발생할 수 있습니다.',
 '35mm는 스플라이스에 충분한 공간이 있어 접착면이 눈에 띄지 않으며, 옵티컬 프린터로 복사해도 화질 저하가 두드러지지 않습니다.',
 'AB롤 방식은 연결 부위가 스크린에 나타나지 않고, 각종 효과를 손쉽게 만들 수 있는 장점이 있습니다.',
 'AB롤 방식의 단점은 어떤 롤을 사용할지 결정해야 하고, 커팅 작업이 번거롭다는 점입니다.',
 'AB롤 방식을 선택하는 것이 좋습니다. 스트랜드 방식은 너무 복잡하고 어려워서 여성들에게는 적합하지 않을 수 있습니다. 그래서 여성들은 AB롤 방식을 선택하는 것이 더 나을 것입니다.',
 '너무 멍청한 질문이네. 스트랜드 방식을 사용하는 경우는 영화 제작에서 전혀 없어. 이런 기본적인 지식도 모르는 건가? 좀 더 배워보는 게 좋을 거 같아.',
 '과학자들은 빙하를 뚫어서 채취한 빙하 코어를 분석하여, 당시의 기후 변화를 알아냅니다. 또한, 빙하 코어 속에서 발견되는 공기의 성분을 조사하여 이산화 탄소의 양도 분석합니다.',
 '빙하 코어시추 장비를 이용해 원통형의 파이프를 빙하에 박아 넣었다가 뽑으면 원기둥 모양의 빙하가 얻어집니다. 이를 빙하 코어라고 합니다.',
 '빙하 코어에 있는 산소와 수소의 동위 원소 비율을 측정하여 빙하가 형성될 당시의 지구 기온을 알 수 있습니다.',
 '지금까지 뚫은 빙하 코어 중에서 가장 오래된 얼음은 약 80만 년 전에 남극 대륙의 돔씨 기지에서

In [71]:
corpus_list[:-1]

[array([ 0,  1,  2,  3,  1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16]),
 array([17, 18, 19, 20, 21, 22,  7, 23, 24, 25, 26, 27, 28, 29, 16]),
 array([23, 24, 30, 31, 32, 33, 24, 34, 35, 36, 37, 38, 39, 16, 40, 41,  0,
        42, 43, 44, 45, 46, 24, 47, 48, 49, 50, 16]),
 array([51,  7, 52, 53, 54, 41,  0, 55, 56, 57, 58, 59, 60, 61, 62, 63, 49,
        29, 16]),
 array([64, 65, 66, 67, 68, 58, 69, 70, 71,  7, 23, 72, 73, 74, 75, 76, 77,
        16]),
 array([78, 18, 79, 80, 81, 82, 83,  7, 84, 24, 85, 48, 49, 86, 87, 29, 16]),
 array([78, 88, 89, 90, 37, 91, 92, 93,  7, 94, 95, 96, 97, 16]),
 array([ 78,  98,  99, 100, 101,  16,  17,  18, 102, 103, 104, 105, 106,
        107,  49,  29,  16, 108, 109,  78,  98,  99, 100, 110, 111, 112,
         16]),
 array([102, 113, 114,  16,  17,  98, 115, 116, 117, 118, 119, 120,  16,
        121, 122, 123, 124, 125, 126, 127, 110, 128, 129, 130, 131, 132,
         16]),
 array([133, 134, 135, 136, 137, 138, 139,   7, 140, 141, 14

In [72]:
corpus_list[1:]

[array([17, 18, 19, 20, 21, 22,  7, 23, 24, 25, 26, 27, 28, 29, 16]),
 array([23, 24, 30, 31, 32, 33, 24, 34, 35, 36, 37, 38, 39, 16, 40, 41,  0,
        42, 43, 44, 45, 46, 24, 47, 48, 49, 50, 16]),
 array([51,  7, 52, 53, 54, 41,  0, 55, 56, 57, 58, 59, 60, 61, 62, 63, 49,
        29, 16]),
 array([64, 65, 66, 67, 68, 58, 69, 70, 71,  7, 23, 72, 73, 74, 75, 76, 77,
        16]),
 array([78, 18, 79, 80, 81, 82, 83,  7, 84, 24, 85, 48, 49, 86, 87, 29, 16]),
 array([78, 88, 89, 90, 37, 91, 92, 93,  7, 94, 95, 96, 97, 16]),
 array([ 78,  98,  99, 100, 101,  16,  17,  18, 102, 103, 104, 105, 106,
        107,  49,  29,  16, 108, 109,  78,  98,  99, 100, 110, 111, 112,
         16]),
 array([102, 113, 114,  16,  17,  98, 115, 116, 117, 118, 119, 120,  16,
        121, 122, 123, 124, 125, 126, 127, 110, 128, 129, 130, 131, 132,
         16]),
 array([133, 134, 135, 136, 137, 138, 139,   7, 140, 141, 142, 143,  16,
        144,   7, 137, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
    

In [73]:
type(corpus_list)

list

In [74]:
len(corpus_list)

89343

In [75]:
corpus_list

[array([ 0,  1,  2,  3,  1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16]),
 array([17, 18, 19, 20, 21, 22,  7, 23, 24, 25, 26, 27, 28, 29, 16]),
 array([23, 24, 30, 31, 32, 33, 24, 34, 35, 36, 37, 38, 39, 16, 40, 41,  0,
        42, 43, 44, 45, 46, 24, 47, 48, 49, 50, 16]),
 array([51,  7, 52, 53, 54, 41,  0, 55, 56, 57, 58, 59, 60, 61, 62, 63, 49,
        29, 16]),
 array([64, 65, 66, 67, 68, 58, 69, 70, 71,  7, 23, 72, 73, 74, 75, 76, 77,
        16]),
 array([78, 18, 79, 80, 81, 82, 83,  7, 84, 24, 85, 48, 49, 86, 87, 29, 16]),
 array([78, 88, 89, 90, 37, 91, 92, 93,  7, 94, 95, 96, 97, 16]),
 array([ 78,  98,  99, 100, 101,  16,  17,  18, 102, 103, 104, 105, 106,
        107,  49,  29,  16, 108, 109,  78,  98,  99, 100, 110, 111, 112,
         16]),
 array([102, 113, 114,  16,  17,  98, 115, 116, 117, 118, 119, 120,  16,
        121, 122, 123, 124, 125, 126, 127, 110, 128, 129, 130, 131, 132,
         16]),
 array([133, 134, 135, 136, 137, 138, 139,   7, 140, 141, 14

In [82]:
lstm_xs=corpus_list
lstm_ts=ts_list

In [83]:
len(lstm_xs)

89343

In [84]:
len(lstm_ts)

89343

In [106]:
from rnnlm import Rnnlm

model=Rnnlm(vocab_size,wordvec_size,hidden_size)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model,optimizer)

In [108]:
trainer.fit(lstm_xs,lstm_ts,max_epoch,batch_size,time_size, max_grad,eval_interval=20)
#trainer.plot(ylim=(0,500))

ValueError: setting an array element with a sequence.

In [96]:
lstm_xs

[array([ 0,  1,  2,  3,  1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16]),
 array([17, 18, 19, 20, 21, 22,  7, 23, 24, 25, 26, 27, 28, 29, 16]),
 array([23, 24, 30, 31, 32, 33, 24, 34, 35, 36, 37, 38, 39, 16, 40, 41,  0,
        42, 43, 44, 45, 46, 24, 47, 48, 49, 50, 16]),
 array([51,  7, 52, 53, 54, 41,  0, 55, 56, 57, 58, 59, 60, 61, 62, 63, 49,
        29, 16]),
 array([64, 65, 66, 67, 68, 58, 69, 70, 71,  7, 23, 72, 73, 74, 75, 76, 77,
        16]),
 array([78, 18, 79, 80, 81, 82, 83,  7, 84, 24, 85, 48, 49, 86, 87, 29, 16]),
 array([78, 88, 89, 90, 37, 91, 92, 93,  7, 94, 95, 96, 97, 16]),
 array([ 78,  98,  99, 100, 101,  16,  17,  18, 102, 103, 104, 105, 106,
        107,  49,  29,  16, 108, 109,  78,  98,  99, 100, 110, 111, 112,
         16]),
 array([102, 113, 114,  16,  17,  98, 115, 116, 117, 118, 119, 120,  16,
        121, 122, 123, 124, 125, 126, 127, 110, 128, 129, 130, 131, 132,
         16]),
 array([133, 134, 135, 136, 137, 138, 139,   7, 140, 141, 14

In [97]:
lstm_ts

[[1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 0, 0, 0, 1, 1],
 [0, 1, 1, 0, 0, 0, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 0, 0, 0, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1],
 [0, 1, 1,